<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_0909LeNet_fashionmnist_tensorboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%config InlineBackend.figure_format = 'retina'
try:
    import bit
except ImportError:
    !pip install ipynbname --upgrade > /dev/null 2>&1
    import os
    import shutil
    if os.path.exists('bit'):
        shutil.rmtree('bit')
    !git clone https://github.com/ShinAsakawa/bit.git
import bit
isColab = bit.isColab
HOME = bit.HOME

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np

# transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

# datasets
trainset = torchvision.datasets.FashionMNIST(
    root='data',
    download=True,
    train=True,
    transform=transform)
testset = torchvision.datasets.FashionMNIST(
    root='data',
    download=True,
    train=False,
    transform=transform)

# dataloaders
trainloader = torch.utils.data.DataLoader(
    trainset, 
    batch_size=4,
    shuffle=True, 
    num_workers=2)

testloader = torch.utils.data.DataLoader(
    testset, 
    batch_size=4,
    shuffle=False, 
    num_workers=2)

# constant for classes
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
           'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

In [3]:
class LeNet(nn.Module):
    """LeNet [@1998LeCun] の実装"""
    def __init__(self):
        super().__init__()

        # 第一畳み込み層の定義
        self.conv1 = nn.Conv2d(in_channels=1, 
                               out_channels=6, 
                               kernel_size=5)
        
        # 最大値プーリング層の定義
        self.pool = nn.MaxPool2d(kernel_size=2, 
                                 stride=2)
        
        self.conv2 = nn.Conv2d(6, 16, 5)  # 第二畳み込み層の定義
        
        # 第一全結合層の定義
        self.fc1 = nn.Linear(in_features=16 * 4 * 4, 
                             out_features=120)
        
        self.fc2 = nn.Linear(120, 84)     # 第二全結合層の定義
        self.fc3 = nn.Linear(84, 10)      # 最終層の定義

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

lenet = LeNet()

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(lenet.parameters(), lr=0.001, momentum=0.9)

In [ ]:
optimizer.__dict__

In [7]:
if not isColab:
    from torch.utils.tensorboard import SummaryWriter

    # default `log_dir` is "runs" - we'll be more specific here
    writer = SummaryWriter('runs/fashion_mnist_experiment_1')

In [ ]:
# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# create grid of images
img_grid = torchvision.utils.make_grid(images)

# helper function to show an image
# (used in the `plot_classes_preds` function below)
def matplotlib_imshow(img, 
                      one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))
        
# show images
matplotlib_imshow(img_grid, one_channel=True)

# write to tensorboard
#writer.add_image('four_fashion_mnist_images', img_grid)

In [9]:
if not isColab:
    !tensorboard --logdir=runs

In [11]:
if not isColab:
    writer.add_graph(net, images)
    writer.close()

In [13]:
# helper function
def select_n_random(data, labels, n=100):
    '''    Selects n random datapoints and their corresponding labels from a dataset    '''
    assert len(data) == len(labels)

    perm = torch.randperm(len(data))
    return data[perm][:n], labels[perm][:n]

# select random images and their target indices
images, labels = select_n_random(trainset.data, trainset.targets)

# get the class labels for each image
class_labels = [classes[lab] for lab in labels]

if not isColab:
    # log embeddings
    features = images.view(-1, 28 * 28)
    writer.add_embedding(features,
                         metadata=class_labels,
                         label_img=images.unsqueeze(1))
    writer.close()

In [ ]:
type(lenet.parameters())
# generator

In [15]:
type(lenet.parameters().__next__())
# torch.nn.parameter.Parameter

torch.nn.parameter.Parameter

In [ ]:
print(lenet.parameters().__next__())
# Parameter containing:
# (0 ,0 ,.,.) = 
#  -0.0998  0.0035 -0.0438 -0.1150 -0.0435
#   0.0310 -0.0750 -0.0405 -0.0745 -0.1095
#  -0.0355  0.0065 -0.0225  0.0729 -0.1114
#   0.0708 -0.0170 -0.0253  0.1060  0.0557
#   0.1057  0.0873  0.0793 -0.0309 -0.0861
#   ...

In [ ]:
%%time
for epoch in range(2):
    # エポックを指定回数だけ繰り返す

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # 第二引数は，スタート位置で，0 なので enumerate(trainloader)と同じ
        # https://docs.python.org/3/library/functions.html#enumerate

        inputs, labels = data        # data は tuple なので分解

        # Variable で包む
        inputs = torch.autograd.Variable(inputs) 
        labels = torch.autograd.Variable(labels)

        # zero クリアする
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = lenet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.detach().numpy()
        
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch+1:2d}, {i+1:5d}] 損失: {running_loss/2000:0.3f}')
            #%
            #      (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0



In [18]:
correct = 0
total = 0
for data in testloader:
    images, labels = data
    outputs = lenet(torch.autograd.Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print(f'検証データセットでの精度: {int(100 * correct / total):3d} %')


検証データセットでの精度:  85 %
